<a href="https://colab.research.google.com/github/NguyenKimLong-20146222/FINAL-PROJECT/blob/main/CODE_FINAL_PROJECT_FACE%20DETECTION%20AND%20COUNTING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nhập những thư viện cần thiết
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tkinter as tk
from PIL import ImageTk, Image

# Tải mô hình nhận diện khuôn mặt đã được huấn luyện trước
model = load_model('modelfinal.h5')

# Định nghĩa kích thước đầu vào
input_shape = (40, 30, 3)

# Khởi tạo bộ phân lớp cascade khuôn mặt
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Hàm để phát hiện và đếm số khuôn mặt
def detect_faces(frame):
    # Chuyển đổi hình ảnh sang ảnh xám
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Sử dụng bộ phân lớp cascade để phát hiện khuôn mặt
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Thay đổi kích thước và tiền xử lý khuôn mặt đã phát hiện
    face_images = []
    for (x, y, w, h) in faces:
        face = cv2.resize(frame[y:y + h, x:x + w], input_shape[:2])
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_images.append(face)
        
        # Vẽ khung xung quanh khuôn mặt
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 6)

    if len(face_images) > 0:
        # Chuyển đổi danh sách hình ảnh khuôn mặt thành mảng numpy và chuẩn hóa
        face_images = np.array(face_images) / 255.0

        # Dự đoán số khuôn mặt sử dụng mô hình nhận diện khuôn mặt
        predictions = model.predict(face_images)

        # Đếm số khuôn mặt đã phát hiện
        face_count = np.sum(predictions[:, 0] >= 1)

        return face_count
    else:
        return 0

# Khởi tạo Tkinter
root = tk.Tk()
root.title("Face Detection")
root.geometry("650x500")

# Tạo một nhãn để hiển thị luồng video
video_label = tk.Label(root)
video_label.pack(pady=10)

# Mở khung bắt video
cap = cv2.VideoCapture(0)

def update_frame():
    ret, frame = cap.read()
    if not ret:
        return

    # Phát hiện khuôn mặt và đếm số khuôn mặt
    face_count = detect_faces(frame)

    # Hiển thị số khuôn mặt trên khung hình
    count_text = str(face_count)
    count_text_size, _ = cv2.getTextSize(count_text, cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    cv2.putText(frame, count_text, (20, frame.shape[0] - 400), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 6)

    # Chuyển đổi khung hình sang định dạng ImageTk
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame)
    image_tk = ImageTk.PhotoImage(image)

    # Cập nhật nhãn video với khung hình mới
    video_label.config(image=image_tk)
    video_label.image = image_tk

    # Lên lịch cho việc cập nhật tiếp theo
    video_label.after(10, update_frame)

# Bắt đầu cập nhật khung hình
update_frame()

# Bắt đầu vòng lặp sự kiện Tkinter
root.mainloop()

# Giải phóng khung bắt video
cap.release()
cv2.destroyAllWindows()